# **Тут я попробовала полностью случайным образом сгенерировать искусственную смесь - очень простым методом (very simple dummy method)**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("default")

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/bostone_gene



In [6]:
cancer_sample_annot = pd.read_csv('/content/drive/MyDrive/bostone_gene/data/Annotation_cell_lines.tsv.gz', sep='\t', index_col=0)
cancer_expr = pd.read_csv('/content/drive/MyDrive/bostone_gene/data/Expression_cell_lines.tsv.gz', sep='\t', index_col=0)

cells_sample_annot = pd.read_csv('/content/drive/MyDrive/bostone_gene/data/Annotation_cells.tsv.gz', sep='\t', index_col=0)
cells_expr = pd.read_csv('/content/drive/MyDrive/bostone_gene/data/Expression_cells.tsv.gz', sep='\t', index_col=0)

# Создадим значения для смешивания:

In [8]:
# for genes:
cancer_genes_lst = list((cancer_expr.T.index).unique())
cell_lst = genes_lst = list((cells_expr.T.index).unique())

gene_lst = cell_lst.copy()  # создаем копию списка cell_lst
gene_lst.extend(cancer_genes_lst)  # добавляем значения из списка cancer_genes_lst в конец списка cell_gene_lst

# for types of cells:
cell_lst = list(cells_sample_annot['general_cell_type'].unique())


# Создадим смесь

In [9]:
num_mixtures = 3000

mixtures = []

for i in range(num_mixtures):
    # randomly select cell types and their weights for the mixture
    cell_types = np.random.choice(cell_lst, size=5, replace=False)
    weights = pd.Series(np.random.dirichlet(np.ones(5)), index=cell_types)

    # generate expression data
    expr = pd.DataFrame()
    for ct in cell_types:
        # randomly select genes and their expression levels for the cell type
        levels = np.random.normal(size=len(genes_lst))
        expr[ct] = pd.Series(levels, index=genes_lst)

    # normalize the expression data by total expression
    total_expr = expr.sum(axis=1)
    expr = expr.divide(total_expr, axis=0)

    # weight the expression data by cell type weights
    expr = expr.multiply(weights, axis=1)
    
    # add the mixture to the list of mixtures
    mixtures.append(expr.sum(axis=1))

# convert the list of mixtures to a DataFrame
mixtures = pd.DataFrame(mixtures, columns=genes_lst)


In [10]:
# create new column with corresponding cell types
mixtures['Cell Type'] = np.random.choice(cell_lst, size=num_mixtures)

# reorder columns - Cell Type is the last column
cols = mixtures.columns.tolist()
cols = cols[:-2] + [cols[-1]] + [cols[-2]]
mixtures = mixtures[cols]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# reorder columns 
cols = mixtures.columns.tolist()
cols = cols[:-2] + [cols[-1]] + [cols[-2]]
mixtures = mixtures[cols]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
mixtures.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Cell Type
0,0.178845,-0.019769,-0.027169,0.127063,0.272393,0.338430,0.140146,0.218110,0.195814,0.215354,...,0.473247,0.243419,0.186983,0.235164,0.179902,0.274056,-0.699537,0.329691,0.055857,Monocytes
1,3.617356,0.945350,0.253615,0.160371,0.151240,0.069230,0.272915,0.249858,-0.638631,0.159945,...,0.112839,-0.036044,0.213022,0.125722,0.180892,0.233416,1.261647,0.339963,0.120471,CD8_T_cells
2,0.191539,-0.006929,0.042412,0.084707,-0.100009,0.195655,0.044769,0.190994,-0.545431,0.271989,...,0.522622,0.075049,0.084826,0.338074,-16.061452,0.197393,0.254652,0.551511,0.181802,T_cells
3,0.313509,0.395097,0.097368,-0.598117,1.548777,0.016697,0.068747,0.446026,-0.028497,0.181677,...,0.553114,0.178166,-0.081034,-0.113417,0.613050,-3.770961,0.408259,0.229951,-0.281517,Monocytes
4,0.004509,0.079548,0.218584,0.342770,0.226957,0.287925,0.194161,1.107869,0.134081,0.124983,...,-0.732494,0.396686,-3.618272,0.413412,0.550753,-1.836736,0.146589,-1.244816,0.237278,NK_cells


# Посмотрим какие данные получились:

In [13]:
mixtures.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,0.004689,-0.030650,0.201298,0.105387,0.332772,0.290733,0.569643,0.291374,0.309994,0.090133,...,0.570393,0.208698,0.322314,0.372611,0.137090,0.145717,0.399087,-0.230648,0.029888,0.015884
std,38.253913,21.200761,3.682106,5.712627,6.634351,3.949031,37.721122,4.950863,5.195590,14.777539,...,23.666830,5.441154,8.116700,15.124940,4.286649,9.072692,10.765537,22.954279,10.077016,6.461391
min,-1714.117024,-1114.847584,-171.506742,-125.484724,-183.214699,-85.462512,-895.005109,-132.561296,-88.583269,-740.185029,...,-98.293053,-154.473418,-293.894684,-457.639632,-184.632139,-395.587067,-193.194283,-1234.788198,-485.905592,-183.051383
25%,0.035194,0.038887,0.058156,0.052959,0.049985,0.047810,0.059688,0.046504,0.056715,0.067591,...,0.051871,0.066934,0.034795,0.075720,0.053934,0.065628,0.050520,0.069413,0.044647,0.053851
50%,0.193647,0.191788,0.205492,0.197514,0.194835,0.200611,0.202452,0.194555,0.203513,0.207593,...,0.199955,0.200661,0.193462,0.206811,0.193306,0.201020,0.195042,0.201275,0.195162,0.193943
75%,0.329306,0.341360,0.340285,0.332510,0.342042,0.352352,0.355894,0.347391,0.347952,0.355880,...,0.349332,0.344833,0.339981,0.357963,0.334031,0.346864,0.343140,0.342541,0.337249,0.343875
max,1192.465774,207.211751,31.873210,180.003771,186.402077,110.977246,1616.722248,192.490348,230.971676,295.524971,...,1283.559102,235.238251,269.129963,659.602059,62.116703,153.862661,500.272941,149.152999,145.318184,174.485531


# Добавим шум с учетом:

- шум считывания (sequencing noise)
- шум технологии (technology noise)
- Poisson noise
- батч-эффект

In [14]:
mixtures.shape

(3000, 20063)

In [ ]:
# Remove zero
mixtures_noised = mixtures.drop(columns=["pk", 'Cell Type'])

# Add noise
sequencing_noise = np.random.normal(scale=0.1, size=mixtures_noised.shape)
technology_noise = np.random.normal(scale=0.05, size=mixtures_noised.shape)
poisson_noise = np.random.poisson(lam=1, size=mixtures_noised.shape)
outlier_noise = np.random.uniform(-0.5, 0.5, size=mixtures_noised.shape)
mixtures_noised += sequencing_noise + technology_noise + poisson_noise + outlier_noise

In [25]:
mixtures_noised.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,0.989224,0.942798,1.211821,1.113507,1.333535,1.279370,1.549846,1.261709,1.300621,1.093539,...,0.791628,1.624064,1.210098,1.330292,1.396386,1.113224,1.125995,1.366679,0.794999,1.073592
std,38.260150,21.203439,3.808625,5.827560,6.734598,4.069718,37.736503,5.073804,5.299112,14.835344,...,15.728581,23.706304,5.547652,8.216296,15.179815,4.439169,9.144628,10.817998,22.946371,10.166923
min,-1713.519084,-1113.169202,-169.143739,-123.704085,-183.325741,-84.808268,-895.499891,-132.716410,-86.913619,-740.164446,...,-837.272950,-98.641927,-153.459820,-294.316799,-457.115880,-185.060116,-396.078441,-192.916262,-1232.218832,-486.067707
25%,0.293453,0.290199,0.351800,0.321680,0.324684,0.327271,0.299510,0.291675,0.334765,0.339692,...,0.351220,0.387639,0.320569,0.292382,0.377807,0.292562,0.368433,0.308271,0.379207,0.332716
50%,1.030724,1.032349,1.094539,1.017597,1.081680,1.068486,1.074287,1.056516,1.058463,1.103411,...,1.046180,1.111983,1.089206,1.065005,1.132926,1.035202,1.055230,1.071135,1.121399,1.099972
75%,1.880197,1.903340,1.895710,1.905078,1.921129,1.901810,1.933312,1.912596,1.929505,1.879979,...,1.891343,1.961360,1.956250,1.941185,2.005299,1.882932,1.881947,1.887309,1.959108,1.991282
max,1192.187508,207.414381,33.312301,181.679906,187.337666,110.777589,1617.587830,193.786906,231.754455,296.726576,...,55.297362,1285.481871,236.477763,271.954528,661.807138,64.057142,155.034555,500.749235,149.754739,148.610226


In [26]:
# Make  positive
mixtures_noised = mixtures_noised.clip(lower=0)


# **Train the model:**

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


x = mixtures_noised
y = mixtures['Cell Type']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy default LogisticRegression : 0.10333333333333333


In [36]:
print("Accuracy default LogisticRegression: {:.2f}%".format(accuracy*100))

Accuracy default LogisticRegression: 10.33%


In [37]:
from sklearn.svm import SVC

svm = SVC(C=0.5, kernel='linear', random_state=2023)

svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_svm)
print("Accuracy of SVM model: {:.2f}%".format(accuracy*100))

Accuracy of SVM model: 10.67%
